In [1]:
#ファイルの読み込み
import pandas as pd
df1 = pd.read_csv('data_jun.csv')
df1.head()

,year,futoko_rate_jun,stu_per_tch_jun,classes_jun,rate_jun,rate_OII_jun,rate_Lve_jun
0,1991,1.04,18.08,3150,4.2,0.004,1.33
1,1992,1.16,17.81,3150,5.0,0.004,1.39
2,1993,1.24,17.43,3150,6.0,0.006,1.40
3,1994,1.32,17.11,3150,8.0,0.004,1.37
4,1995,1.42,16.86,3150,8.8,0.004,1.37


In [3]:
#特徴量から'year', ’stu_per_tch_jun’, 'rate_Lve_jun'を削除
df2 = df1.drop(['year','stu_per_tch_jun', 'rate_Lve_jun'], axis = 1)

#標準化する
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(df2)
sc_df1 = sc.transform(df2)
sc_df1 = pd.DataFrame(sc_df1, columns = df2.columns)
sc_df1.head()

,futoko_rate_jun,classes_jun,rate_jun,rate_OII_jun
0,-1.849284,1.176697,-1.459897,-0.962980
1,-1.709489,1.176697,-1.220285,-0.962980
2,-1.616292,1.176697,-0.920769,-0.696441
3,-1.523095,1.176697,-0.321738,-0.962980
4,-1.406598,1.176697,-0.082125,-0.962980


In [4]:
#多項式特徴量、交互作用特徴量を出してみる
from sklearn.preprocessing import PolynomialFeatures
sc_df2 = sc_df1.loc[:, 'classes_jun':]
poly = PolynomialFeatures(degree = 2)
sc_df3 = poly.fit_transform(sc_df2)
poly.get_feature_names_out()

array(['1', 'classes_jun', 'rate_jun', 'rate_OII_jun', 'classes_jun^2',
       'classes_jun rate_jun', 'classes_jun rate_OII_jun', 'rate_jun^2',
       'rate_jun rate_OII_jun', 'rate_OII_jun^2'], dtype=object)

In [5]:
#先ほど出した多項式特徴量、交互作用特徴量をデータフレームにする
sc_df4 = pd.DataFrame(sc_df3, columns = ['1', 'classes_jun', 'rate_jun', 'rate_OII_jun', 'classes_jun^2',
       'classes_jun rate_jun', 'classes_jun rate_OII_jun', 'rate_jun^2',
       'rate_jun rate_OII_jun', 'rate_OII_jun^2'])
sc_df4.head()

,1,classes_jun,rate_jun,rate_OII_jun,classes_jun^2,classes_jun rate_jun,classes_jun rate_OII_jun,rate_jun^2,rate_jun rate_OII_jun,rate_OII_jun^2
0,1.0,1.176697,-1.459897,-0.962980,1.384615,-1.717856,-1.133136,2.131300,1.405852,0.927331
1,1.0,1.176697,-1.220285,-0.962980,1.384615,-1.435905,-1.133136,1.489095,1.175110,0.927331
2,1.0,1.176697,-0.920769,-0.696441,1.384615,-1.083466,-0.819500,0.847816,0.641261,0.485030
3,1.0,1.176697,-0.321738,-0.962980,1.384615,-0.378588,-1.133136,0.103515,0.309827,0.927331
4,1.0,1.176697,-0.082125,-0.962980,1.384615,-0.096637,-1.133136,0.006745,0.079085,0.927331


In [6]:
#'futoko_rate_jun’を戻して'1'を削除
sc_df4['futoko_rate_jun'] = sc_df1['futoko_rate_jun']
sc_df4 = sc_df4.drop(['1'], axis = 1)
sc_df4.head()


,classes_jun,rate_jun,rate_OII_jun,classes_jun^2,classes_jun rate_jun,classes_jun rate_OII_jun,rate_jun^2,rate_jun rate_OII_jun,rate_OII_jun^2,futoko_rate_jun
0,1.176697,-1.459897,-0.962980,1.384615,-1.717856,-1.133136,2.131300,1.405852,0.927331,-1.849284
1,1.176697,-1.220285,-0.962980,1.384615,-1.435905,-1.133136,1.489095,1.175110,0.927331,-1.709489
2,1.176697,-0.920769,-0.696441,1.384615,-1.083466,-0.819500,0.847816,0.641261,0.485030,-1.616292
3,1.176697,-0.321738,-0.962980,1.384615,-0.378588,-1.133136,0.103515,0.309827,0.927331,-1.523095
4,1.176697,-0.082125,-0.962980,1.384615,-0.096637,-1.133136,0.006745,0.079085,0.927331,-1.406598


In [7]:
#相関係数を出してみる
abs(sc_df4.corr()['futoko_rate_jun']).sort_values(ascending = False)

futoko_rate_jun             1.000000
rate_OII_jun                0.813922
rate_jun rate_OII_jun       0.749921
rate_OII_jun^2              0.577614
classes_jun^2               0.547418
classes_jun                 0.476923
classes_jun rate_OII_jun    0.473606
classes_jun rate_jun        0.222681
rate_jun                    0.059723
rate_jun^2                  0.055788
Name: futoko_rate_jun, dtype: float64

In [8]:
#特徴量と正解データに分割
col = ['rate_OII_jun', 'classes_jun^2']
x = sc_df4[col]
t = sc_df4['futoko_rate_jun']

In [9]:
#訓練データとテストデータの分割
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,t, test_size = 0.2, random_state = 0)

#モデルを作成し学習させる
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train,y_train) 

LinearRegression()

In [10]:
#決定係数を出してみる
model.score(x_test, y_test)

0.628722094108893